In [1]:
import os
import datetime
import warnings
import polars as pl
import pandas as pd
import altair as alt

from src.najdi_rok import najdi_rok
from src.pocet_stran import pocet_stran
from src.bez_bordelu import bez_bordelu
from src.alt_friendly import alt_friendly
from src.hezke_jmeno import hezke_jmeno
from src.kristi_promin import kristi_promin
from src.zjisti_vazbu import zjisti_vazbu

pl.Config(tbl_rows=100)
alt.data_transformers.disable_max_rows()
alt.themes.register('irozhlas', kristi_promin)
alt.themes.enable('irozhlas')
warnings.filterwarnings('ignore')

In [2]:
df = pl.read_parquet(os.path.join("data/cnb_sloupce","leader.parquet"))
df = df.join(pl.read_parquet(os.path.join("data/cnb_sloupce","100.parquet")), left_on="001", right_on="001", how="left")
df = df.join(pl.read_parquet(os.path.join("data/cnb_sloupce","008.parquet")), left_on="001", right_on="001", how="left")
df = df.to_pandas()
df = df[df["leader"].str[6].isin(["a", "t"])]
df = df[~df["leader"].str[7].isin(["b", "i", "s", " "])]
df = df[(df["008"].str[15:17] == "xr") & (df["008"].str[35:38] == "cze")]
df = pl.from_pandas(df)
df = df.join(pl.read_parquet(os.path.join("data/cnb_sloupce","020.parquet")), left_on="001", right_on="001", how="left")
df = df.join(pl.read_parquet(os.path.join("data/cnb_sloupce","022.parquet")), left_on="001", right_on="001", how="left")
df = df.join(pl.read_parquet(os.path.join("data/cnb_sloupce","245.parquet")), left_on="001", right_on="001", how="left")
df = df.join(pl.read_parquet(os.path.join("data/cnb_sloupce","300.parquet")), left_on="001", right_on="001", how="left")
df = df.join(pl.read_parquet(os.path.join("data/cnb_sloupce","655.parquet")), left_on="001", right_on="001", how="left")
df = df.join(pl.read_parquet(os.path.join("data/cnb_sloupce","700.parquet")), left_on="001", right_on="001", how="left")
df = df.explode("022_a").filter(pl.col("022_a").is_null())
df = df.with_columns(pl.col('008').map_elements(najdi_rok, return_dtype=int).alias('rok'))
df = df.with_columns(pl.col('300_a').map_elements(pocet_stran, return_dtype=int).alias('stran'))
df = df.with_columns(pl.col('245_a').map_elements(bez_bordelu, return_dtype=str))
df = df.explode("020_q").with_columns(pl.col("020_q").map_elements(zjisti_vazbu, return_dtype=str).alias('vazba'))
df = df.explode('245_p').with_columns(pl.col('245_p').map_elements(bez_bordelu, return_dtype=str))
print(len(df))
df = df.filter(~pl.col('rok').is_null()).sort(by='rok')
df = df.filter((~pl.col("245_h").str.contains("grafika")) | pl.col("245_h").is_null())
print(len(df))

1001279
991612


In [3]:
df = df.with_columns(pl.col('100_a').map_elements(hezke_jmeno, return_dtype=str).alias('jmeno'))

In [4]:
bio = pl.read_parquet(os.path.join('data','narozeni-umrti-gender.parquet'))

In [5]:
df = df.join(bio, on='100_7', how='left')

In [6]:
df.filter(pl.col('stran') >= 50).with_columns(pl.col(pl.String).str.replace("Kytice z pověstí národních","Kytice")).group_by(
    ['jmeno','245_a','245_p']
).len().sort(by='len',descending=True).head(30).join(
    df.with_columns(pl.col(pl.String).str.replace("Kytice z pověstí národních","Kytice")).group_by(['jmeno','245_a']).agg(pl.col('rok').min()
), on=['245_a','jmeno'], how='left')

jmeno,245_a,245_p,len,rok
str,str,str,u32,i64
null,"""Bible""",null,255,null
"""Božena Němcová""","""Babička""",null,223,1855
null,"""Divadelní ochotník""",null,216,null
"""Karel Hynek Mácha""","""Máj""",null,162,1872
"""Karel Jaromír Erben""","""Kytice""",null,154,1853
null,"""Čítanka pro školy obecné""",null,110,null
"""Alois Jirásek""","""Staré pověsti české""",null,99,1894
"""Jan Karafiát""","""Broučci""",null,96,1876
null,"""Russkij jazyk""",null,96,null


In [7]:
topka = [
    ["Božena Němcová","Babi"],
    ["Karel Hynek Mácha","Má[jg]"],
    ["Karel Jaromír Erben","(?i)Kyti"],
    ["Alois Jirásek","(?i)pověst"],
    ["Jan Neruda","(?i)malostrans"],
    ["Alois Jirásek","Psohlavci"],
    ["Božena Němcová","(?i)pohorsk"],
    ["Antoine de Saint-Exupéry","Malý princ"],
    ["Karel Tůma","Z českých mlýnů"],
    ["Karel Čapek","(?i)válka s mloky"],
    ["Alexandre Dumas","Tři mušketýři"],
    ["Karel Poláček","Bylo nás pět"],
    ["Petr Bezruč","Slezské písně"],
    ["Jaroslav Hašek", "Švejk"]
]

In [8]:
len(topka)

14

In [9]:
top_vyber = pl.DataFrame()
for autor_titul in topka:
    vyber = df.filter(~pl.col("100_7").is_null()).filter((pl.col("jmeno").str.contains(autor_titul[0])) & (pl.col('245_a').str.contains(autor_titul[1])))
    pravy_titul = vyber.sort(by='rok').select(pl.col('245_a')).to_series().to_list()[0].split(' - ')[0].replace('a jiné podivné historky','').strip()
    pocet = len(vyber)
    hezky_nazev = f"{pocet}× {autor_titul[0].split(' ')[-1]}: {pravy_titul}"
    top_vyber = pl.concat(
        [top_vyber, vyber.with_columns(
            pl.lit(hezky_nazev).alias('kniha')).with_columns(
            pl.lit(pocet).alias('kolik')
        ).select(pl.col(['rok','kniha','kolik','umrti','jmeno']))
        ])

top_vyber = top_vyber.with_columns(pl.col('umrti').map_elements(
            lambda x: datetime.date(year=int(x), month=1, day=1), return_dtype=pl.Date
        )
        .cast(pl.Datetime))
poradi_vyber = top_vyber.group_by('kniha').len().sort(by='len',descending=True).select(pl.col('kniha')).to_series().to_list()
poradi_pocet = top_vyber.group_by('kniha').len().sort(by='len',descending=True).select(pl.col('len')).to_series().to_list()
print(poradi_vyber)

['242× Němcová: Babička', '242× Mácha: Mág', '198× Erben: Kytice z pověstí národních', '120× Neruda: Povídky malostranské', '116× Hašek: Dobrý voják Švejk', '100× Jirásek: Staré pověsti české', '97× Dumas: Tři mušketýři', '71× Němcová: Pohorská vesnice', '66× Jirásek: Psohlavci', '56× Saint-Exupéry: Malý princ', '54× Bezruč: Slezské písně', '52× Tůma: Z českých mlýnů', '51× Čapek: Válka s Mloky', '48× Poláček: Bylo nás pět']


In [10]:
top_vyber.sample(10)

rok,kniha,kolik,umrti,jmeno
i64,str,i32,datetime[μs],str
1914,"""71× Němcová: Pohorská vesnice""",71,1862-01-01 00:00:00,"""Božena Němcová"""
2004,"""198× Erben: Kytice z pověstí n…",198,1870-01-01 00:00:00,"""Karel Jaromír Erben"""
1941,"""198× Erben: Kytice z pověstí n…",198,1870-01-01 00:00:00,"""Karel Jaromír Erben"""
1919,"""242× Mácha: Mág""",242,1836-01-01 00:00:00,"""Karel Hynek Mácha"""
1918,"""97× Dumas: Tři mušketýři""",97,1870-01-01 00:00:00,"""Alexandre Dumas"""
2016,"""48× Poláček: Bylo nás pět""",48,1945-01-01 00:00:00,"""Karel Poláček"""
1967,"""242× Němcová: Babička""",242,1862-01-01 00:00:00,"""Božena Němcová"""
1972,"""198× Erben: Kytice z pověstí n…",198,1870-01-01 00:00:00,"""Karel Jaromír Erben"""
1923,"""97× Dumas: Tři mušketýři""",97,1870-01-01 00:00:00,"""Alexandre Dumas"""


In [11]:
base = alt.Chart(
    alt_friendly(top_vyber),
    title=alt.Title(
        ['Kdy vycházely nejvydávanější knihy'],
        subtitle = ['Růžové tečky jsou knihy původní české, modré překladové.','Černá čárka označuje rok úmrtí autora nebo autorky.']
    ),
width=280)

kdy_umreli = alt.Chart(alt_friendly(top_vyber)).mark_tick(
    color='#292829',  # optional: you can specify color
    thickness=1.5,
    height=9
).encode(
    x=alt.X('umrti:T', title=None),
    y=alt.Y("kniha:N", sort=poradi_vyber, title=None, axis=alt.Axis(orient='left', domainOpacity=0, tickColor='white', labelExpr='split(datum.label, "× ")[1]')))

domaci = base.mark_circle(size=8).encode(
    x=alt.X('rok:T', title=None), 
    y=alt.Y('kniha:N', sort=poradi_vyber, title=None, axis=alt.Axis(orient='left', labels=False)),
    yOffset=alt.YOffset("jitter:Q", scale=alt.Scale(range=[3, 15])),
    color=alt.Color('kniha:N', scale=alt.Scale(range=['#E09DA3']), legend=None)
).transform_filter(
    (alt.datum.kniha != '97× Dumas: Tři mušketýři') & (alt.datum.kniha != '56× Saint-Exupéry: Malý princ')
).transform_calculate(jitter="sqrt(-2*log(random()))*cos(2*PI*random())")

zahranicni = base.mark_circle(size=8, color='#81A9D5').encode(
    x=alt.X('rok:T', title=None), 
    y=alt.Y('kniha:N', sort=poradi_vyber, title=None, axis=alt.Axis(orient='left', labels=False)),
    yOffset=alt.YOffset("jitter:Q", scale=alt.Scale(range=[3, 15])),
    color=alt.Color('kniha:N', scale=alt.Scale(range=['#81A9D5']), legend=None)
).transform_filter(
    (alt.datum.kniha == '97× Dumas: Tři mušketýři') | (alt.datum.kniha == '56× Saint-Exupéry: Malý princ')
).transform_calculate(jitter="sqrt(-2*log(random()))*cos(2*PI*random())")

pocty = base.mark_circle(size=0).encode(x=alt.X('rok:T', title=None), 
    y=alt.Y('kniha:N', sort=poradi_pocet, title=None, axis=alt.Axis(orient="right", tickColor='white', labelExpr='split(datum.label, " ")[0]')))

nejvydavanejsi = alt.layer(domaci, zahranicni, kdy_umreli, pocty).configure_view(stroke='transparent').resolve_scale(color='independent',x="shared").resolve_axis(y='independent')

nejvydavanejsi

alt.LayerChart(...)

In [12]:
import os
import json
from src.me_to_neurazi import me_to_neurazi
with open(os.path.join('src','kredity.json'), 'r', encoding='utf-8') as kredity:
    kredity = json.loads(kredity.read())

In [13]:
me_to_neurazi(nejvydavanejsi, kredity=kredity['default'], soubor='01_nejvydavanejsi_knihy')

<figure><a href="https://data.irozhlas.cz/knihy-grafy/01_nejvydavanejsi_knihy.svg" target="_blank"><img src="https://data.irozhlas.cz/knihy-grafy/01_nejvydavanejsi_knihy.svg" width="100%" alt="Omlouváme se, ale alternativní text se nepodařilo vygenerovat. Texty v grafu by měly být čitelné ze zdrojového souboru SVG." /></a></figure>
<figure><a href="https://michalkasparek.cz/sklad/01_nejvydavanejsi_knihy.svg" target="_blank"><img src="https://michalkasparek.cz/sklad/01_nejvydavanejsi_knihy.svg" width="100%" alt="Omlouváme se, ale alternativní text se nepodařilo vygenerovat. Texty v grafu by měly být čitelné ze zdrojového souboru SVG." /></a></figure>


In [14]:
df = df.with_columns(pl.col('245_p').map_elements(bez_bordelu, return_dtype=str))

In [15]:
df.group_by("245_p").len().sort(by="len",descending=True)

245_p,len
str,u32
null,933441
"""Místopis Moravy""",107
"""Geometrie""",89
"""Vlastní státní správa""",87
"""Evropa""",79
"""Matematika""",73
"""Zvířata""",70
"""Text""",67
"""Textová část""",61


In [16]:
df.group_by("245_a").agg(pl.col("245_p").unique().len()).sort(by="245_p",descending=True).head(100)

245_a,245_p
str,u32
"""Sčítání lidu, domů a bytů k 1.…",307
"""Pedagogické dokumenty""",291
"""Učební dokumenty pro učební ob…",287
"""Učební dokumenty pro střední o…",271
"""Prozatimní učební dokumenty uč…",249
"""Učební plán a osnovy odborných…",248
"""Z pamětí amerického detektiva""",145
"""Přednášky Československé spole…",129
"""Star Wars""",112
